In [80]:
from clickhouse_connect import get_client
import pandas as pd
import numpy as np
import logging
import pytz
from datetime import datetime, timedelta, timezone, date
from truedata.analytics import TD_analytics
from truedata.history import TD_hist

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

IST = pytz.timezone("Asia/Kolkata")
IST = timezone(timedelta(hours=5, minutes=30))

# Config (ClickHouse | TrueData)

In [46]:
# Clickhouse
host = "localhost"
port = 8123
username = "ingest_w"
password = "ingest_secret"
database = "market"

# TrueData
td_username = "True9030"
td_password = "vineet@9030"
td_log_level = logging.WARNING

# Connect to ClickHouse | TrueData Analytics

In [47]:
def get_clickhouse_conn(host,port,username,password,database):
    try:
        conn = get_client(
            host=host,
            port=port,
            username=username,
            password=password,
            database=database
        )
        conn.query("SELECT 1")
        print("✅ Connected to ClickHouse database!")
        print(f"📊 Database: {conn.database}")
        print(f"🕐 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        return conn
    except Exception as e:
        return f"Exception in get_clickhouse_conn as : {e}"

In [48]:
conn = get_clickhouse_conn(host,port,username,password,database)
obj_td_analytics = TD_analytics(td_username, td_password, log_level=td_log_level)

✅ Connected to ClickHouse database!
📊 Database: market
🕐 Current time: 2025-11-08 15:50:07


(2025-11-08 15:50:08,387) WARNING :: Connected successfully to TrueData Anaytical Data Service...  (PID:26452 Thread:19628)
(2025-11-08 15:50:08,387) WARNING :: Connected successfully to TrueData Anaytical Data Service...  (PID:26452 Thread:19628)
(2025-11-08 15:50:08,387) WARNING :: Connected successfully to TrueData Anaytical Data Service...  (PID:26452 Thread:19628)
(2025-11-08 15:50:08,387) WARNING :: Connected successfully to TrueData Anaytical Data Service...  (PID:26452 Thread:19628)


# Functions (Option Chain | OI Trigger | OI Trap)

In [77]:

#! HELPING FUNCTIONS

def add_oi_trigger(df, threshold_percent=10, threshold_absolute=1000):
    """
    CORRECTED OI Trigger column based on significant OI changes
    Now properly handles mixed scripts/strikes/expiries by grouping correctly
    
    Args:
        df: DataFrame with option data
        threshold_percent: Percentage threshold for OI change trigger (default: 10%)
        threshold_absolute: Absolute threshold for OI change trigger (default: 1000 contracts)
    
    Returns:
        DataFrame with oi_trigger column added
    """
    
    # Initialize oi_trigger column
    df['oi_trigger'] = 'NORMAL'
    
    # Calculate absolute OI change
    df['oi_change_abs'] = df['oi_change'].abs()
    
    # Apply trigger logic
    # Trigger if either percentage change > threshold OR absolute change > threshold
    trigger_conditions = (
        (df['oi_change_perc'].abs() > threshold_percent) |  # More than 10% change
        (df['oi_change_abs'] > threshold_absolute)          # More than 1000 contracts change
    )
    
    # Set trigger types based on direction
    df.loc[trigger_conditions & (df['oi_change'] > 0), 'oi_trigger'] = 'OI_BUILDUP'
    df.loc[trigger_conditions & (df['oi_change'] < 0), 'oi_trigger'] = 'OI_UNWIND'
    
    # Add intensity levels
    high_intensity = (
        (df['oi_change_perc'].abs() > threshold_percent * 2) |  # More than 20% change
        (df['oi_change_abs'] > threshold_absolute * 2)           # More than 2000 contracts change
    )
    
    df.loc[high_intensity & (df['oi_change'] > 0), 'oi_trigger'] = 'HIGH_OI_BUILDUP'
    df.loc[high_intensity & (df['oi_change'] < 0), 'oi_trigger'] = 'HIGH_OI_UNWIND'
    
    # Clean up temporary column
    df = df.drop('oi_change_abs', axis=1)
    
    return df

def add_oi_trap(df, high_oi_threshold=0.6, significant_oi_change=5):
    """
    CORRECTED oi_trap column based on Call/Put Writers Trap logic
    Now properly handles mixed scripts/strikes/expiries by grouping correctly
    
    OI Trap Types:
    - Call Writers Trap: High call OI + OI decreasing (writers trapped)
    - Put Writers Trap: High put OI + OI decreasing (writers trapped)
    - Call Writers Trap-Pentry: High call OI + OI increasing (new bullish positions)
    - Put Writers Trap-Pentry: High put OI + OI increasing (new bearish positions)
    - Call Writers Trap-Pexit: High call OI + OI increasing (unwinding)
    - Put Writers Trap-Pexit: High put OI + OI increasing (unwinding)
    - No Trap: No significant trap detected
    
    Args:
        df: DataFrame with option data
        high_oi_threshold: Threshold for high OI (default: 0.6) (60th percentile within the same underlying)
        significant_oi_change: Threshold for significant OI change (default: 5%)

    Returns:
        DataFrame with oi_trap column added
    """
    
    # Initialize oi_trap column
    df['oi_trap'] = 'No Trap'
    
    # Sort by timestamp for proper analysis
    df = df.sort_values(['underlying', 'expiry', 'strike', 'side', 'timestamp_ist']).copy()
    
    # CORRECTED: Calculate OI percentiles per underlying and trading session
    # This ensures we compare OI within the same underlying, not across different scripts
    df['oi_percentile'] = df.groupby(
        ['underlying', 'expiry', 'strike', 'side', df['timestamp_ist'].dt.date]
    )['oi'].transform(lambda x: x.rank(pct=True))
    
    print(f"📊 CORRECTED Trap Analysis:")
    print(f"High OI threshold: {high_oi_threshold} (per underlying)")
    print(f"Significant OI change: {significant_oi_change}%")
    print(f"OI percentile range: {df['oi_percentile'].min():.3f} to {df['oi_percentile'].max():.3f}")
    print(f"OI change % range: {df['oi_change_perc'].min():.2f}% to {df['oi_change_perc'].max():.2f}%")
    
    #! Call Writers Trap Logic
    call_mask = df['side'] == 'C'
    
    #! Call Writers Trap: High call OI + OI decreasing (writers trapped)
    call_trap_conditions = (
        call_mask &
        (df['oi_percentile'] > high_oi_threshold) &
        (df['oi_change'] < 0) &  # OI decreasing (writers trapped)
        (df['oi_change_perc'] < -significant_oi_change)  # Significant decrease
    )
    df.loc[call_trap_conditions, 'oi_trap'] = 'Call Writers Trap'
    
    #! Call Writers Trap-Pentry: High call OI + OI increasing (new bullish positions)
    call_entry_conditions = (
        call_mask &
        (df['oi_percentile'] > high_oi_threshold) &
        (df['oi_change'] > 0) &  # OI increasing (new positions)
        (df['oi_change_perc'] > significant_oi_change)  # Significant increase
    )
    df.loc[call_entry_conditions, 'oi_trap'] = 'Call Writers Trap-Pentry'
    
    #! Call Writers Trap-Pexit: High call OI + OI increasing (unwinding)
    call_exit_conditions = (
        call_mask &
        (df['oi_percentile'] > high_oi_threshold) &
        (df['oi_change'] > 0) &  # OI increasing (unwinding)
        (df['oi_change_perc'] > significant_oi_change) &  # Significant increase
        (df['oi_change_perc'] < significant_oi_change * 2)  # But not too extreme
    )
    df.loc[call_exit_conditions, 'oi_trap'] = 'Call Writers Trap-Pexit'
    
    #! Put Writers Trap Logic
    put_mask = df['side'] == 'P'
    
    #! Put Writers Trap: High put OI + OI decreasing (writers trapped)
    put_trap_conditions = (
        put_mask &
        (df['oi_percentile'] > high_oi_threshold) &
        (df['oi_change'] < 0) &  # OI decreasing (writers trapped)
        (df['oi_change_perc'] < -significant_oi_change)  # Significant decrease
    )
    df.loc[put_trap_conditions, 'oi_trap'] = 'Put Writers Trap'
    
    #! Put Writers Trap-Pentry: High put OI + OI increasing (new bearish positions)
    put_entry_conditions = (
        put_mask &
        (df['oi_percentile'] > high_oi_threshold) &
        (df['oi_change'] > 0) &  # OI increasing (new positions)
        (df['oi_change_perc'] > significant_oi_change)  # Significant increase
    )
    df.loc[put_entry_conditions, 'oi_trap'] = 'Put Writers Trap-Pentry'
    
    #! Put Writers Trap-Pexit: High put OI + OI increasing (unwinding)
    put_exit_conditions = (
        put_mask &
        (df['oi_percentile'] > high_oi_threshold) &
        (df['oi_change'] > 0) &  # OI increasing (unwinding)
        (df['oi_change_perc'] > significant_oi_change) &  # Significant increase
        (df['oi_change_perc'] < significant_oi_change * 2)  # But not too extreme
    )
    df.loc[put_exit_conditions, 'oi_trap'] = 'Put Writers Trap-Pexit'
    
    #! Show oi_trap detection statistics
    oi_trap_counts = df['oi_trap'].value_counts()
    print(f"\n🎯 Traps Detection Summary:")
    for oi_trap, count in oi_trap_counts.items():
        print(f"{oi_trap}: {count} records")
    
    #! Clean up temporary columns
    df = df.drop(['oi_percentile'], axis=1)
    
    return df

def add_delta_change(df, delta_change_threshold=0.01, delta_velocity_threshold=0.001):
    """
    Add Δ-Delta (Intraday Change in Delta) Feature
    
    This tracks how Delta changes over time for each unique option contract
    Each unique combination of (underlying, strike, expiry, side) is treated as a separate script
    
    Args:
        df: DataFrame with option data
        delta_change_threshold: Threshold for Delta change (default: 0.01) (1% Delta change)
        delta_velocity_threshold: Threshold for Delta velocity (default: 0.001) (0.1% per minute)
    
    Returns:
        DataFrame with Δ-Delta columns added
    """
    
    # Sort by unique option contract and timestamp
    df = df.sort_values(['underlying', 'strike', 'expiry', 'side', 'timestamp_ist']).copy()
    
    # Calculate Δ-Delta for each unique option contract
    df['delta_change'] = df.groupby(['underlying', 'strike', 'expiry', 'side'])['delta'].diff()
    
    # Calculate Δ-Delta percentage
    df['delta_change_perc'] = df.groupby(['underlying', 'strike', 'expiry', 'side'])['delta'].pct_change() * 100
    
    # Calculate Delta velocity (change per minute)
    # how many minutes passed since the last data point
    df['time_diff_minutes'] = df.groupby(['underlying', 'strike', 'expiry', 'side'])['timestamp_ist'].diff().dt.total_seconds() / 60

    # Calculate Delta velocity (change per minute)
    df['delta_velocity'] = df['delta_change'] / df['time_diff_minutes']
    
    # Add Delta momentum categories
    df['delta_momentum'] = 'STABLE'
    
    # Accelerating: Delta change is increasing
    accelerating_conditions = (
        (df['delta_change'].abs() > delta_change_threshold) &
        (df['delta_velocity'].abs() > delta_velocity_threshold) &
        (df['delta_change'] > 0)
    )
    df.loc[accelerating_conditions, 'delta_momentum'] = 'ACCELERATING'
    
    # Decelerating: Delta change is decreasing
    decelerating_conditions = (
        (df['delta_change'].abs() > delta_change_threshold) &
        (df['delta_velocity'].abs() > delta_velocity_threshold) &
        (df['delta_change'] < 0)
    )
    df.loc[decelerating_conditions, 'delta_momentum'] = 'DECELERATING'
    
    # Show Δ-Delta statistics
    print(f"\n📈 Δ-Delta Feature Statistics:")
    print(f"Delta change range: {df['delta_change'].min():.4f} to {df['delta_change'].max():.4f}")
    print(f"Delta change % range: {df['delta_change_perc'].min():.2f}% to {df['delta_change_perc'].max():.2f}%")
    print(f"Delta velocity range: {df['delta_velocity'].min():.6f} to {df['delta_velocity'].max():.6f}")
    
    delta_momentum_counts = df['delta_momentum'].value_counts()
    print(f"\n🎯 Delta Momentum Distribution:")
    for momentum, count in delta_momentum_counts.items():
        print(f"{momentum}: {count} records")
    
    # Clean up temporary columns
    df = df.drop(['time_diff_minutes'], axis=1)
    
    return df

#! MAIN

def get_master_features( underlying, start_date, expiry=None, limit=None, latest_only=False, strikes=None):
    """
        Get (Option Chain + OI Trigger + OI Trap + Δ-Delta) features

        Args:
            underlying (str): Stock/index symbol (e.g., 'NIFTY', 'BANKNIFTY')
            start_date (str): Start date in 'YYYY-MM-DD' format
            expiry (str, optional): Expiry date as string 'YYYY-MM-DD'. If None, include all expiries.
            limit (int, optional): Max number of records to return. If None, return all records.
            latest_only (bool, optional): If True, return only the most recent timestamp. Default is False.
            strikes (list[int], optional): List of strikes to include. If None, include all strikes.
        
        Returns:
            pandas.DataFrame: Standardized features ready for ML
    """

    # Base WHERE clause
    where_clause = f"""
        oc.snap_minute_ist >= '{start_date}'
        AND oc.underlying = '{underlying}'
        AND oc.delta IS NOT NULL
        AND oc.iv IS NOT NULL
        AND oc.ltp > 0
    """

    # Add expiry filter only if provided
    if expiry is not None:
        where_clause += f" AND oc.expiry = '{expiry}'"

    # Add strike filter only if provided
    if strikes is not None and len(strikes) > 0:
        strike_list = ', '.join(map(str, strikes))
        where_clause += f" AND oc.strike IN ({strike_list})"

    # Build the base query
    query = f"""
    SELECT 
        -- Standardized timestamp (IST)
        oc.snap_minute_ist as timestamp_ist,

        -- Option Chain Features
        oc.underlying,
        oc.expiry,
        oc.strike,
        oc.side,
        oc.ltp,
        oc.bid_price,
        oc.ask_price,
        oc.volume,
        oc.oi,
        oc.prev_oi,

        -- Greeks (Delta, IV, etc.)
        oc.delta,
        oc.iv,
        oc.theta,
        oc.vega,
        oc.gamma,
        oc.rho,

        -- Calculated OI Change
        oc.oi - oc.prev_oi as oi_change,

        -- OI Change Percentage
        CASE 
            WHEN oc.prev_oi > 0 THEN ((oc.oi - oc.prev_oi) / oc.prev_oi) * 100
            ELSE NULL 
        END as oi_change_perc

    FROM market.option_chain_1m oc
    WHERE {where_clause}
    ORDER BY oc.snap_minute_ist ASC, oc.expiry, oc.strike, oc.side
    """

    # Add LIMIT only if provided
    if limit is not None:
        query += f"\nLIMIT {limit}"

    df = conn.query_df(query)

    # Add OI Trigger logic
    df = add_oi_trigger(df)

    # Add Trap logic
    df = add_oi_trap(df)

    # Add Δ-Delta feature
    df = add_delta_change(df)

    print(f"✅ Retrieved {len(df)} standardized records for {underlying}")
    if expiry:
        print(f"📅 Filtered for expiry: {expiry}")
    if strikes:
        print(f"🎯 Filtered for strikes: {strikes}")
    if limit:
        print(f"🔢 Limited to {limit} rows")
    print(f"📊 Added OI change calculations, oi_trigger, oi_trap, and Δ-Delta columns")

    # If latest_only is True, filter for the most recent timestamp
    if latest_only and not df.empty:
        latest_timestamp = df['timestamp_ist'].max()
        df = df[df['timestamp_ist'] == latest_timestamp]
        print(f"📅 Filtered for latest timestamp: {latest_timestamp}")

    return df


# Functions (VIX)

In [ ]:
def get_strike_step(symbol):
    if "BANKNIFTY" in symbol:
        return 100
    elif "NIFTY" in symbol:
        return 50
    elif "FINNIFTY" in symbol:
        return 50
    return 1


def _fetch_iv_by_expiry_today(conn,SYMBOL):

    today = date.today().isoformat()

    #! 1. Get all rows for today and SYMBOL
    q = '''
        SELECT snap_minute_ist, expiry, strike, side, iv
        FROM option_chain_1m
        WHERE underlying = %(symbol)s
          AND toDate(snap_minute_ist) = %(today)s
        ORDER BY expiry, snap_minute_ist, strike
    '''
    rows = list(conn.query(q, parameters={"symbol": SYMBOL, "today": today}).named_results())
    if not rows:
        return pd.DataFrame(columns=["time","expiry_ts","expiry_str","tenor_tag","atm_strike","ce_iv","pe_iv","iv_avg"])
    df = pd.DataFrame(rows)
    df['expiry_str'] = df['expiry'].astype(str)
    df['expiry_ts'] = pd.to_datetime(df['expiry']).astype(int) // 10**9

    #! 2. Find the two nearest expiries (sorted ascending)
    expiry_dates = sorted(df['expiry'].unique())
    if len(expiry_dates) < 1:
        return pd.DataFrame(columns=["time","expiry_ts","expiry_str","tenor_tag","atm_strike","ce_iv","pe_iv","iv_avg"])
    near_expiry = expiry_dates[0]
    near_dt = pd.to_datetime(near_expiry)
    # Find all expiries in the next month after near expiry's month
    far_month_expiries = [e for e in expiry_dates if (pd.to_datetime(e).year > near_dt.year) or (pd.to_datetime(e).year == near_dt.year and pd.to_datetime(e).month == near_dt.month + 1)]
    if far_month_expiries:
        far_month = pd.to_datetime(far_month_expiries[0]).month
        far_year = pd.to_datetime(far_month_expiries[0]).year
        # Get all expiries in that month
        far_expiries_in_month = [e for e in expiry_dates if pd.to_datetime(e).year == far_year and pd.to_datetime(e).month == far_month]
        far_expiry = max(far_expiries_in_month)
        expiry_map = {near_expiry: 'near', far_expiry: 'far'}
    else:
        expiry_map = {near_expiry: 'near'}

    #! 3. Get spot LTP and strike step for ATM calculation
    ul_ltp = float(obj_td_analytics.get_spot_ltp(SYMBOL).LTP.iloc[0])
    strike_step = get_strike_step(SYMBOL)

    #! 4. For each expiry, for each minute, get ATM strike and IVs
    out_rows = []
    for expiry, tenor_tag in expiry_map.items():
        df_exp = df[df['expiry'] == expiry].copy()
        # For each minute
        for minute, group in df_exp.groupby(df_exp['snap_minute_ist']):
            # Find ATM strike for this minute
            strikes = group['strike'].unique()
            if len(strikes) == 0:
                continue
            # ATM = closest to ul_ltp, snapped to strike step
            atm_strike = min(strikes, key=lambda s: abs(float(s) - ul_ltp))
            # Get CE/PE IVs for ATM
            ce_iv = group[(group['strike'] == atm_strike) & (group['side'] == 'C')]['iv']
            pe_iv = group[(group['strike'] == atm_strike) & (group['side'] == 'P')]['iv']
            ce_iv = float(ce_iv.iloc[0]) if not ce_iv.empty else None
            pe_iv = float(pe_iv.iloc[0]) if not pe_iv.empty else None
            iv_avg = 0.5 * (ce_iv + pe_iv) if ce_iv is not None and pe_iv is not None else None
            out_rows.append({
                'time': pd.to_datetime(minute).strftime('%H:%M'),
                'expiry_ts': int(pd.to_datetime(expiry).timestamp()),
                'expiry_str': str(expiry),
                'tenor_tag': tenor_tag,
                'atm_strike': atm_strike,
                'ce_iv': ce_iv,
                'pe_iv': pe_iv,
                'iv_avg': iv_avg
            })
    out_df = pd.DataFrame(out_rows)
    out_df[['ce_iv', 'pe_iv', 'iv_avg']] = out_df[['ce_iv', 'pe_iv', 'iv_avg']] * 100
    return out_df.sort_values(['time','tenor_tag']).reset_index(drop=True)


def _compute_vix30(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    today = datetime.now(IST).date()
    df = df.copy()
    df["t_dt"] = pd.to_datetime(df["time"], format="%H:%M").map(lambda t: datetime.combine(today, t.time(), tzinfo=IST))
    df["expiry_dt"] = pd.to_datetime(df["expiry_ts"], unit="s", utc=True).dt.tz_convert(IST)

    out = []
    T30 = 30.0/365.0
    for t, blk in df.groupby("t_dt", sort=True):
        blk = blk.copy()
        blk["days_left"] = (blk["expiry_dt"] - t).dt.total_seconds()/86400.0
        near = blk[blk["days_left"].between(0, 30, inclusive="left")].sort_values("days_left").head(1)
        far  = blk[blk["days_left"] >= 30].sort_values("days_left").head(1)

        vix = None
        if not near.empty and not far.empty:
            iv1 = float(near["iv_avg"].iloc[0]) / 100.0
            iv2 = float(far["iv_avg"].iloc[0])  / 100.0
            T1  = float(near["days_left"].iloc[0]) / 365.0
            T2  = float(far["days_left"].iloc[0])  / 365.0
            V1, V2 = iv1*iv1*T1, iv2*iv2*T2
            w1 = (T2 - T30) / (T2 - T1) if T2 != T1 else 1.0
            V30 = w1*V1 + (1.0 - w1)*V2
            vix = 100.0*np.sqrt(V30 / T30)
        else:
            use = near if not near.empty else (far if not far.empty else None)
            if use is not None and not use.empty:
                iv = float(use["iv_avg"].iloc[0]) / 100.0
                T  = float(use["days_left"].iloc[0]) / 365.0
                if T > 0:
                    vix = 100.0 * iv * np.sqrt(T / T30)

        if vix is not None:
            out.append({"t_dt": t, "time": t.strftime("%H:%M"), "vix30": round(float(vix), 2)})

    print(
        f"{t:%H:%M} near_iv%={near['iv_avg'].iloc[0]:.2f} "
        f"near_days={near['days_left'].iloc[0]:.1f}  "
        f"far_iv%={(far['iv_avg'].iloc[0] if not far.empty else float('nan')):.2f} "
        f"far_days={(far['days_left'].iloc[0] if not far.empty else float('nan')):.1f}"
    )

    return pd.DataFrame(out).sort_values("t_dt")


def _resample_vixdf(vixdf: pd.DataFrame, interval_min: int) -> pd.DataFrame:
    if vixdf.empty or int(interval_min) <= 1:
        return vixdf
    d = vixdf.copy()
    # ensure we have a datetime index
    if "t_dt" not in d.columns:
        today = datetime.now(IST).date()
        d["t_dt"] = pd.to_datetime(d["time"], format="%H:%M").map(
            lambda t: datetime.combine(today, t.time(), tzinfo=IST)
        )
    d = d.set_index("t_dt").sort_index()
    # last value per bucket
    out = d.resample(f"{int(interval_min)}min")[["vix30"]].last().dropna(how="all").reset_index()
    out["time"] = out["t_dt"].dt.strftime("%H:%M")
    return out[["t_dt", "time", "vix30"]]


def fetch_vix_for_indices(conn, indices, interval_min):
    all_vixdfs = []  # to store vix dataframes for each index

    for index in indices:
        rows = _fetch_iv_by_expiry_today(conn, index)
        if not rows.empty:
            vixdf = _compute_vix30(rows)
            vixdf = _resample_vixdf(vixdf, interval_min)

            # Add instrument column
            vixdf['instrument'] = index

            # Rename t_dt to date (optional — or keep as is)
            vixdf['date'] = pd.to_datetime(vixdf['t_dt']).dt.date
            vixdf['time'] = pd.to_datetime(vixdf['t_dt']).dt.strftime('%H:%M')

            # Keep only required columns
            vixdf = vixdf[['date', 'time', 'instrument', 'vix30']]

            all_vixdfs.append(vixdf)

    # Combine all instruments into one DataFrame
    if all_vixdfs:
        final_df = pd.concat(all_vixdfs, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame(columns=['date', 'time', 'instrument', 'vix30'])

# Dataset [master(oc,traps,triggers,chng_in_delta) | VIX | FII_DII]

Master

In [78]:
df_master = get_master_features(underlying='NIFTY', start_date='2025-11-03', expiry=None, limit=None, latest_only=False, strikes=None)
df_master

📊 CORRECTED Trap Analysis:
High OI threshold: 0.6 (per underlying)
Significant OI change: 5%
OI percentile range: 0.003 to 1.000
OI change % range: -79.01% to 15563.44%

🎯 Traps Detection Summary:
No Trap: 710789 records
Call Writers Trap-Pentry: 69653 records
Put Writers Trap-Pentry: 68545 records
Put Writers Trap: 14627 records
Call Writers Trap: 11049 records
Put Writers Trap-Pexit: 7479 records
Call Writers Trap-Pexit: 5831 records

📈 Δ-Delta Feature Statistics:
Delta change range: -0.3764 to 0.1787
Delta change % range: -100.00% to inf%
Delta velocity range: -0.114790 to 0.118250

🎯 Delta Momentum Distribution:
STABLE: 880052 records
ACCELERATING: 4424 records
DECELERATING: 3497 records
✅ Retrieved 887973 standardized records for NIFTY
📊 Added OI change calculations, oi_trigger, oi_trap, and Δ-Delta columns


,timestamp_ist,underlying,expiry,strike,side,ltp,bid_price,ask_price,volume,oi,prev_oi,delta,iv,theta,vega,gamma,rho,oi_change,oi_change_perc,oi_trigger,oi_trap,delta_change,delta_change_perc,delta_velocity,delta_momentum
1180,2025-11-03 10:30:00+05:30,NIFTY,2026-06-30,12000.00,P,20.0,1.050000,20.049999,0,325,325,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,NORMAL,No Trap,NaN,NaN,NaN,STABLE
2455,2025-11-03 10:31:00+05:30,NIFTY,2026-06-30,12000.00,P,20.0,1.050000,20.049999,0,325,325,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,NORMAL,No Trap,NaN,NaN,NaN,STABLE
3730,2025-11-03 10:32:00+05:30,NIFTY,2026-06-30,12000.00,P,20.0,1.050000,20.049999,0,325,325,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,NORMAL,No Trap,NaN,NaN,NaN,STABLE
5006,2025-11-03 10:33:00+05:30,NIFTY,2026-06-30,12000.00,P,20.0,1.050000,20.049999,0,325,325,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,NORMAL,No Trap,NaN,NaN,NaN,STABLE
6282,2025-11-03 10:34:00+05:30,NIFTY,2026-06-30,12000.00,P,20.0,1.050000,20.049999,0,325,325,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,NORMAL,No Trap,NaN,NaN,NaN,STABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883242,2025-11-07 13:52:00+05:30,NIFTY,2026-12-29,31000.00,C,259.0,255.050003,269.000000,3900,35625,35250,0.16174,0.11764,-1.88372,0.62766,0.00008,53.846230,375,1.063830,NORMAL,No Trap,0.00000,0.000000,0.00000,STABLE
884415,2025-11-07 13:53:00+05:30,NIFTY,2026-12-29,31000.00,C,260.0,255.050003,260.000000,3975,35700,35250,0.16247,0.11734,-1.88824,0.62994,0.00008,54.085751,450,1.276596,NORMAL,No Trap,0.00073,0.451338,0.00073,STABLE
885588,2025-11-07 13:54:00+05:30,NIFTY,2026-12-29,31000.00,C,260.0,255.050003,260.000000,3975,35775,35250,0.16247,0.11734,-1.88824,0.62994,0.00008,54.085751,525,1.489362,NORMAL,No Trap,0.00000,0.000000,0.00000,STABLE
886761,2025-11-07 13:55:00+05:30,NIFTY,2026-12-29,31000.00,C,260.0,255.000000,283.000000,4050,35775,35250,0.16245,0.11735,-1.88809,0.62987,0.00008,54.077412,525,1.489362,NORMAL,No Trap,-0.00002,-0.012308,-0.00002,STABLE


VIX

In [79]:
indices = ["NIFTY", "BANKNIFTY"]
interval_min = 1

df_vix = fetch_vix_for_indices(conn, indices, interval_min)
df_vix

13:56 near_iv%=8.16 near_days=2.6  far_iv%=11.35 far_days=51.6
13:56 near_iv%=12.25 near_days=16.6  far_iv%=11.89 far_days=51.6


,date,time,instrument,vix30
0,2025-11-08,10:33,NIFTY,11.03
1,2025-11-08,10:34,NIFTY,11.02
2,2025-11-08,10:35,NIFTY,11.05
3,2025-11-08,10:36,NIFTY,11.05
4,2025-11-08,10:37,NIFTY,11.04
...,...,...,...,...
223,2025-11-08,13:52,BANKNIFTY,12.07
224,2025-11-08,13:53,BANKNIFTY,12.09
225,2025-11-08,13:54,BANKNIFTY,12.05
226,2025-11-08,13:55,BANKNIFTY,12.05


FII_DII

In [63]:
df_fii_dii = pd.read_csv("fii_dii\\data\\2_net_output_oi_data.csv", parse_dates=["Date"])
df_fii_dii

,Date,CLIENT_TYPE,Future Index Long,Future Index Short,Future Stock Long,Future Stock Short,Option Index Call Long,Option Index Put Long,Option Index Call Short,Option Index Put Short,...,Total Positions carried 6 Days Ago Stock Call,Total Positions carried 7 Days Ago Stock Call,Total Positions carried Today Stock Put,Total Positions carried 1 Day Ago Stock Put,Total Positions carried 2 Days Ago Stock Put,Total Positions carried 3 Days Ago Stock Put,Total Positions carried 4 Days Ago Stock Put,Total Positions carried 5 Days Ago Stock Put,Total Positions carried 6 Days Ago Stock Put,Total Positions carried 7 Days Ago Stock Put
0,2012-01-02,Client,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-03,Client,-18749.0,24323.0,-324.0,15131.0,-44829.0,108677.0,8185.0,52011.0,...,20186.0,20186.0,14680.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0
2,2012-01-04,Client,-16855.0,19443.0,-4001.0,17467.0,-24088.0,117868.0,23701.0,56410.0,...,20186.0,20186.0,17064.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0
3,2012-01-05,Client,-15556.0,30577.0,831.0,28144.0,-8056.0,150702.0,48628.0,88704.0,...,20186.0,20186.0,19290.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0
4,2012-01-06,Client,-11779.0,32283.0,14456.0,28934.0,15663.0,149744.0,51529.0,100663.0,...,20186.0,20186.0,18409.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0,9672.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13520,2025-10-30,FII,-1506.0,10628.0,-25600.0,34489.0,152903.0,88194.0,180577.0,13522.0,...,-25844.0,-24499.0,28818.0,23138.0,21942.0,52611.0,61934.0,59392.0,49079.0,49422.0
13521,2025-10-31,FII,-456.0,7769.0,-1271.0,19084.0,162636.0,56112.0,129213.0,38343.0,...,-12793.0,-25844.0,38763.0,28818.0,23138.0,21942.0,52611.0,61934.0,59392.0,49079.0
13522,2025-11-03,FII,368.0,6522.0,-4153.0,14205.0,-54349.0,31680.0,-53169.0,52820.0,...,-12433.0,-12793.0,41913.0,38763.0,28818.0,23138.0,21942.0,52611.0,61934.0,59392.0
13523,2025-11-04,FII,1272.0,11663.0,-25249.0,47366.0,-264639.0,-141917.0,-266144.0,-166135.0,...,-14899.0,-12433.0,52528.0,41913.0,38763.0,28818.0,23138.0,21942.0,52611.0,61934.0


# Merge

In [66]:
import pandas as pd

# -------------------------------
# 1️⃣ Base DataFrame (already combined)
# Option Chain + Delta + Traps + Triggers
# -------------------------------
# df_master = your main DataFrame
# Columns assumed: ['timestamp_ist', 'underlying', 'expiry', 'strike', 'side', 'delta', 'trap', 'trigger', ...]

# -------------------------------
# 2️⃣ Load VIX DataFrame
# -------------------------------
# df_vix = your VIX DataFrame
# Columns assumed: ['timestamp_ist', 'underlying', 'vix_value']

# -------------------------------
# 3️⃣ Load FII/DII DataFrame
# -------------------------------
# df_fii_dii = your FII/DII DataFrame
# Columns assumed: ['date', 'fii_net_buy', 'dii_net_buy']

# ===============================
# 🧹 Step A — Clean & Normalize
# ===============================
# --- Fix column names
df_master.rename(columns={'symbol': 'underlying'}, inplace=True)
df_vix.rename(columns={'symbol': 'underlying'}, inplace=True)

# --- Normalize 'underlying'
df_master["underlying"] = df_master["underlying"].astype(str).str.upper().str.strip()
df_vix["underlying"] = df_vix["underlying"].astype(str).str.upper().str.strip()

# --- Fix datetime columns
df_master["timestamp_ist"] = pd.to_datetime(df_master["timestamp_ist"])
df_vix["timestamp_ist"] = pd.to_datetime(df_vix["timestamp_ist"])

# --- Sort before merging (required for merge_asof)
df_master.sort_values(["underlying", "timestamp_ist"], inplace=True)
df_vix.sort_values(["underlying", "timestamp_ist"], inplace=True)

# ===============================
# 🧩 Step B — Merge VIX (minute-level)
# ===============================

# --- Fix timezone mismatch
if df_master["timestamp_ist"].dt.tz is not None:
    df_master["timestamp_ist"] = df_master["timestamp_ist"].dt.tz_localize(None)
if df_vix["timestamp_ist"].dt.tz is not None:
    df_vix["timestamp_ist"] = df_vix["timestamp_ist"].dt.tz_localize(None)

# --- Align precision (seconds)
df_master["timestamp_ist"] = pd.to_datetime(df_master["timestamp_ist"]).dt.floor("s")
df_vix["timestamp_ist"] = pd.to_datetime(df_vix["timestamp_ist"]).dt.floor("s")

# --- Sort before merge
df_master.sort_values(["underlying", "timestamp_ist"], inplace=True)
df_vix.sort_values(["underlying", "timestamp_ist"], inplace=True)

# --- Merge
df_master = pd.merge_asof(
    df_master,
    df_vix,
    by="underlying",
    on="timestamp_ist",
    direction="backward",
    tolerance=pd.Timedelta("2min")
)

# ===============================
# 💰 Step C — Merge FII/DII (daily-level)
# ===============================
# Align daily dates
df_master["date"] = df_master["timestamp_ist"].dt.date
df_fii_dii["date"] = pd.to_datetime(df_fii_dii["date"]).dt.date

df_master = df_master.merge(df_fii_dii, on="date", how="left")

# ===============================
# 🧮 Step D — Clean Final Dataset
# ===============================
# Optional: Drop duplicates or missing timestamps
df_master.drop_duplicates(subset=["underlying", "timestamp_ist", "strike", "side"], inplace=True)
df_master.reset_index(drop=True, inplace=True)

# ===============================
# ✅ Final Output
# ===============================
print("✅ Combined ML-ready dataset shape:", df_master.shape)
print(df_master.head(5))


MergeError: incompatible merge keys [1] dtype('<M8[s]') and dtype('<M8[ns]'), must be the same type

# Rough

In [60]:
# Ensure column names match
df_master.rename(columns={'symbol': 'underlying'}, inplace=True)
df_vix.rename(columns={'symbol': 'underlying'}, inplace=True)
